<a href="https://colab.research.google.com/github/avinashkarkada/GROMACS-COLAB/blob/main/PL_CHARMgui_GROMACS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Make sure you are connected to a GPU node

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
#@title Download and unzip the compressed folder of GROMACS 2020.6 version
!wget https://raw.githubusercontent.com/pb3lab/ibm3202/master/software/gromacs.tar.gz
!tar xzf gromacs.tar.gz

In [ ]:
!pip install cmake --upgrade

In [ ]:
%%bash
source /content/gromacs/bin/GMXRC
pwd
gmx -h

In [ ]:
#@title Make a folder in google drive and enter its name. Paste the contents of your gromacs input folder in that folder.

import os
from pathlib import Path 
Folder_name = "" #@param {type:"string"}
mdpath = Path("/content/" + Folder_name + "/") 

if os.path.exists(mdpath):
  print("path already exists")
if not os.path.exists(mdpath):
  os.mkdir(mdpath)
  print("path was succesfully created")

os.chdir(mdpath)
!pwd

In [ ]:
from google.colab import drive
import os
from pathlib import Path

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Google_Drive_Path = '/content/drive/MyDrive/' #@param {type:"string"}
workDir = Google_Drive_Path


In [ ]:
!cp -r $workDir/* $mdpath


In [ ]:
!pwd
!chmod a+x *
!ls -l

In [ ]:
#@title Make sure that you have changed the commmands below according to your files.

!/content/gromacs/bin/gmx grompp -f step4.0_minimization.mdp -o step4.0_minimization.tpr -c step3_input.gro -r step3_input.gro -p topol.top -n index.ndx -maxwarn 1
!/content/gromacs/bin/gmx mdrun -v -deffnm step4.0_minimization
!/content/gromacs/bin/gmx grompp -f step4.1_equilibration.mdp -o step4.1_equilibration.tpr -c step4.0_minimization.gro -r step3_input.gro -p topol.top -n index.ndx
!/content/gromacs/bin/gmx mdrun -v -deffnm step4.1_equilibration


!/content/gromacs/bin/gmx grompp -f step5_production.mdp -o step5_production_10ns.tpr -c step4.1_equilibration.gro -p topol.top -n index.ndx
!/content/gromacs/bin/gmx mdrun -v -deffnm step5_production_10ns


In [ ]:
#@title Analysis
%%bash
source /content/gromacs/bin/GMXRC
printf '1\n1' | gmx trjconv -s step5_production_10ns.tpr -f step5_production_10ns.xtc -o step5_production_10ns.pdb -pbc cluster
printf '1\n1' | gmx trjconv -s step5_production_10ns.tpr -f step5_production_10ns.xtc -o protein-0-10ns.xtc -pbc cluster
printf '4\n4' | gmx rms -s step5_production_10ns.tpr -f step5_production_10ns.pdb -o protein-0-1-RMSD.xvg 
gmx rmsf -s step5_production_10ns.tpr -f step5_production_10ns.pdb -o protein-0-10ns-RMSF.xvg -res <<< 3

In [ ]:
#@title Run this to copy the entire project to your drive. This will paste all the results in "result" in your original drive folder.
#Warning!!!!
#Check if you have enough space!
!mkdir $Google_Drive_Path/result
!cp -r $mdpath/* $Google_Drive_Path/result